In [485]:
import psycopg2
from psycopg2 import OperationalError
from psycopg2.extensions import register_adapter, AsIs
import numpy as np
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.bool_, psycopg2._psycopg.AsIs)
import json
import pandas as pd

In [486]:
#get credentials
with open("postgre_credentials.json") as file:
    data = json.load(file)

In [487]:
#template
connection = psycopg2.connect(
    database = data["db_name"], 
    user = data["db_user"],
    password = data["db_password"],
    host = data["db_host"],
    port = data["db_port"]
)


In [488]:
cur = connection.cursor()

In [489]:
query = "DROP TABLE anime CASCADE"
cur.execute(query)
connection.commit()

In [490]:
query="""CREATE TABLE IF NOT EXISTS anime (
anime_id INTEGER PRIMARY KEY,
eng_name TEXT,
type TEXT,
source TEXT,
episode_count INTEGER,
maturity_rating TEXT,
MAL_score DECIMAL,
scored_count INTEGER,
MAL_rank INTEGER,
MAL_popularity INTEGER,
favorite_count INTEGER,
studio_name TEXT,
licensor TEXT,
air_status TEXT,
is_airing BOOLEAN,
year INTEGER,
season TEXT
)
"""

In [491]:
cur.execute(query)


In [492]:
connection.commit()

In [493]:
query="""CREATE TABLE IF NOT EXISTS date (
date_id SERIAL PRIMARY KEY,
year INTEGER,
month INTEGER,
day INTEGER
)
"""
cur.execute(query)
connection.commit()

In [494]:
query="""CREATE TABLE IF NOT EXISTS show_name (
alternate_name TEXT PRIMARY KEY,
anime_id TEXT REFERENCES anime (anime_id)
)
"""
cur.execute(query)
connection.commit()

In [495]:
query="""CREATE TABLE IF NOT EXISTS genre (
genre_id SERIAL PRIMARY KEY,
genre_name TEXT,
anime_id INTEGER REFERENCES anime (anime_id)
)
"""
cur.execute(query)
connection.commit()

In [496]:
query="""CREATE TABLE IF NOT EXISTS google_trends (
google_trends_id SERIAL PRIMARY KEY,
show_name TEXT REFERENCES show_name(alternate_name),
date INTEGER REFERENCES date(date_id),
popularity_score INT
)
"""
cur.execute(query)
connection.commit()

In [497]:
#Loop through MAL csv file
#insert into anime table first
#split alternative names and loop to insert each show name
#split through genres and insert each genre

In [498]:
def split_column(row, col):
    row_attributes = mal_df.loc[row,col]
    split_row = row_attributes.replace(" ", "").split(",")
    return split_row

In [499]:
def insert_genre(split_row, anime_id):
    for genre in split_row:
        cur.execute("""INSERT INTO genre(genre_id, genre_name, anime_id)
        VALUES (%s,%s)""",
        (genre, anime_id))
        connection.commit()

In [500]:
def insert_anime_info(row):
    try:
        time = row["premiered"].split(" ")
        year = time[1]
        season = time[0]
        #print(year, season)
        cur.execute("""INSERT INTO anime(anime_id, eng_name, type, source,episode_count,
                    maturity_rating, MAL_score, scored_count, MAL_rank, MAL_popularity,
                    favorite_count, studio_name, licensor, air_status, is_airing, year, season)
                    VALUES (%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",
                   (row["anime_id"],row["title_english"],row["type"], row["source"], row["episodes"],
                   row["rating"], row["score"], row["scored_by"], row["rank"], row["popularity"],
                   row["favorites"], row["studio"], row["licensor"], row["status"], row["airing"],
                   year, season))
        connection.commit()
    except: 
        time = row["aired_string"].split("to")[0].split(" ")
        year = int(time[2])
        season = seasons[time[0]]
        cur.execute("""INSERT INTO anime(anime_id, eng_name, type, source,episode_count,
                    maturity_rating, MAL_score, scored_count, MAL_rank, MAL_popularity,
                    favorite_count, studio_name, licensor, air_status, is_airing, year, season)
                    VALUES (%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",
                   (row["anime_id"],row["title_english"],row["type"], row["source"], row["episodes"],
                   row["rating"], row["score"], row["scored_by"], row["rank"], row["popularity"],
                   row["favorites"], row["studio"], row["licensor"], row["status"], row["airing"],
                   year, season))
        connection.commit()
        

    


In [501]:
#test with 1 anime
with open("Anime_List_Path.txt") as file:
    path = file.read()
    
mal_df = pd.read_csv(path)

In [502]:
seasons = {"Jan":"Winter",
          "Feb": "Winter",
          "Mar": "Spring",
          "Apr": "Spring",
          "May": "Spring",
          "Jun": "Summer",
          "Jul": "Summer",
          "Aug": "Summer",
          "Sep": "Fall",
          "Oct": "Fall",
          "Nov": "Fall",
          "Dec": "Winter"}

In [503]:
#insert_anime_info(mal_df.iloc[0])

for idx,row in mal_df.iterrows():
    insert_anime_info(mal_df.iloc[idx])

NumericValueOutOfRange: integer out of range


In [504]:
mal_df.loc[55,"anime_id"]

33185

In [484]:
connection.close()